In [2]:
from preamble import *
HTML('''<style>html, body{overflow-y: visible !important} .CodeMirror{min-width:100% !important;} .rise-enabled .CodeMirror, .rise-enabled .output_subarea{font-size:100%; line-height:1.0; overflow: visible;} .output_subarea pre{width:100%}</style>''') # For slides
#HTML('''<style>html, body{overflow-y: visible !important} .output_subarea{font-size:100%; line-height:1.0; overflow: visible;}</style>''') # For slides
InteractiveShell.ast_node_interactivity = "all"

## Agenda

- Introduction and Motivation
- Artificial Neuron
- Gradient Descent
- Backpropagation
- Perceptron
- Multilayered Perceptron
- **MLP Classification**

- Convolutional Neural Networks
- Recurrent Neural Networks

### Mnist dataset
![mnist-digits](images/mnistdigits.gif)

## Classification output Softmax

- We would like to output the probability distribution over a set of classes
- We need a output neuron for each class, such that each value corresponds to 
    - $P(y=i|\mathbf{x})$ for computing the $i$ class with the $i$-th neuron
* Softmax units -> Multinoulli output distributions
    * multiple neurons output the probability of each class
    * Normalized with the softmax function
    * $p(y=j|x)=\frac{e^{\mathbf{x}^\top w_j}} {\sum^{n}_{k=1}{e^{\mathbf{x}^\top w}}}$
    * strictly postivie
    * sums to one

### Dataset

- 784 dimensional vector $x$ for each datapoint 
- 10 dimensional vector, $y$ for output 
    - represents: probability distribution over the classes

### Design decisions

- Output activation function
- Hiddel layer activation function
- Architecture of the model
    - Number of hidden layers
    - Number neurons per layer

- Loss function
    - Properties of the loss function: differentiable (smooth)
    - Monotonically increasing with the distance from the target value   

### Design decisions

- Output activation function : Softmax
    - $p(y=j|x)=\frac{e^{\mathbf{x}^\top w_j}} {\sum^{n}_{k=1}{e^{\mathbf{x}^\top w}}}$
- Hiddel layer activation function: RelU
- Architecture of the model
    - Number of hidden layers : 2
    - Number neurons per layer: 256

- Loss function: Cross entropy
    - $L(\mathbf{x}, y; \theta) = - \sum_c 1_{(y=c)} \log\ o_c = -log\ o_y$

## MNIST classifier 
### Tensorflow implementation


In [3]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [5]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [6]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 185.524865254
Epoch: 0002 cost= 42.128311551
Epoch: 0003 cost= 26.393205963
Epoch: 0004 cost= 18.311323359
Epoch: 0005 cost= 13.360680003
Epoch: 0006 cost= 9.956092792
Epoch: 0007 cost= 7.374592196
Epoch: 0008 cost= 5.557482387
Epoch: 0009 cost= 4.100462057
Epoch: 0010 cost= 3.013828182
Epoch: 0011 cost= 2.287341650
Epoch: 0012 cost= 1.761813821
Epoch: 0013 cost= 1.393472588
Epoch: 0014 cost= 1.003515057
Epoch: 0015 cost= 0.885189935
Optimization Finished!
Accuracy: 0.9491
